In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load the dataset
df = pd.read_csv('creditcard.csv')
X = df.iloc[:, 1:-1].values
y = df.iloc[:, -1].values


In [3]:

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [4]:


# SOM parameters
map_size = (10, 10)  # SOM grid size
input_len = X.shape[1]  # Number of features
sigma = 1.0  # Neighborhood radius
learning_rate = 0.5  # Learning rate
epochs = 100  # Number of training epochs


In [5]:

# Initialize SOM weights
som_weights = tf.Variable(tf.random.normal((map_size[0], map_size[1], input_len)))

# Training function
def train_som(X, som_weights, epochs, sigma, learning_rate):
    for epoch in range(epochs):
        for sample in X:
            # Find the Best Matching Unit (BMU)
            distances = tf.reduce_sum(tf.square(som_weights - sample), axis=2)
            bmu_index = tf.argmin(distances)
            bmu = tf.unravel_index(bmu_index.numpy(), map_size)

            # Update weights
            for i in range(map_size[0]):
                for j in range(map_size[1]):
                    # Calculate the neighborhood function
                    distance_to_bmu = tf.sqrt(tf.reduce_sum(tf.square(tf.cast([i, j], tf.float32) - tf.cast(bmu, tf.float32))))
                    neighborhood = tf.exp(-tf.square(distance_to_bmu) / (2 * tf.square(sigma)))

                    # Update the weight
                    som_weights[i, j].assign(som_weights[i, j] + learning_rate * neighborhood * (sample - som_weights[i, j]))

        # Decay learning rate and sigma
        learning_rate *= 0.99
        sigma *= 0.99

    return som_weights


In [7]:

# Train the SOM
som_weights = train_som(X, som_weights, epochs, sigma, learning_rate)

# Visualization
def plot_som(som_weights):
    plt.figure(figsize=(10, 10))
    plt.pcolor(som_weights.numpy().mean(axis=2), cmap='bone')
    plt.colorbar()
    plt.show()

plot_som(som_weights)

InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [2] vs. [2,10] [Op:Sub] name: 